<a href="https://colab.research.google.com/github/Shankar-Hadimani/ai-ml-engineering/blob/master/notebook/03_sentiment_anlaysis_AutoNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### install automl for NLP
!pip install autoviml

     |████████████████████████████████| 122kB 5.8MB/s 
     |████████████████████████████████| 148.9MB 73kB/s 
     |████████████████████████████████| 66.3MB 127kB/s 
     |████████████████████████████████| 327kB 43.4MB/s 
     |████████████████████████████████| 174kB 40.2MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 133kB 33.1MB/s 
     |████████████████████████████████| 6.8MB 42.1MB/s 
  Created wheel for shap: filename=shap-0.37.0-cp36-cp36m-linux_x86_64.whl size=463901 sha256=c8a184bd21ea7838917eda0cc18baf5244e68e9be20d84f35401203dcc7d0dda
  Stored in directory: /root/.cache/pip/wheels/df/ad/b0/aa7815ec68850d66551ef618095eccb962c8f6022f1d3dd989
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=7d1c9d55949426bc7cb2b4278927c352372c3492eeca45a3d29d5480fad15df0
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built shap emoji
  F

In [2]:
# set up libraries
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd


In [3]:
dataset, info = tfds.load('amazon_us_reviews/Personal_Care_Appliances_v1_00', with_info=True, batch_size=-1)
train_dataset = dataset['train']

Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Personal_Care_Appliances_v1_00/0.1.0.incomplete5DS6DB/amazon_us_reviews-train.tfrecord


Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Personal_Care_Appliances_v1_00/0.1.0. Subsequent calls will reuse this data.


In [4]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [5]:
### convert tensor to numpy array
dataset = tfds.as_numpy(train_dataset)

In [6]:
dataset

{'data': {'customer_id': array([b'13986323', b'50574716', b'50593972', ..., b'40719682',
         b'35596948', b'29430209'], dtype=object),
  'helpful_votes': array([0, 3, 0, ..., 0, 0, 0], dtype=int32),
  'marketplace': array([b'US', b'US', b'US', ..., b'US', b'US', b'US'], dtype=object),
  'product_category': array([b'Personal_Care_Appliances', b'Personal_Care_Appliances',
         b'Personal_Care_Appliances', ..., b'Personal_Care_Appliances',
         b'Personal_Care_Appliances', b'Personal_Care_Appliances'],
        dtype=object),
  'product_id': array([b'B00847JQZ6', b'B00N5HD340', b'B0077L1X24', ..., b'B000UZ8X2W',
         b'B000NURPPK', b'B001EY5GNW'], dtype=object),
  'product_parent': array([b'997683625', b'955577225', b'120764066', ..., b'96066145',
         b'58591097', b'986877728'], dtype=object),
  'product_title': array([b'SE - Reading Glass - Spring Loaded Hinges, 4.0x - RTS62400',
         b'Straight Razor',
         b'Philips Sonicare Flexcare & Healthy White Plastic

In [7]:
### extract required fields from the dataset
helpful_votes = dataset['data']['helpful_votes']
review_headline = dataset['data']['review_headline']
review_body = dataset['data']['review_body']
rating = dataset['data']['star_rating']

In [8]:
### convert above features to a pandas dataframe
reviews_df = pd.DataFrame(np.hstack((helpful_votes[:,None],review_headline[:,None],review_body[:,None],rating[:,None])),
                      columns=['votes','headline','reviews','rating']
                      )

## Define schema 
schema_dict = {'votes': int, 'headline':str, 'reviews':str, 'rating':int }

In [9]:
### apply schema onto pandas dataframe
reviews_df = reviews_df.astype(schema_dict)

#reviews_df

In [10]:
### create an additonal feature as 'target', if rarting is greater or equal to 4
reviews_df['target'] = reviews_df['rating'].apply(lambda x: 1 if x >= 4 else 0)

In [13]:
# reviews_df
reviews_df.shape

(85981, 5)

In [14]:
### view rarting distribution. 
### Seems like imbalanced dataset ( rating distributed)  
reviews_df["target"].value_counts()

1    62554
0    23427
Name: target, dtype: int64

In [15]:
### verufy the schema assigne
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85981 entries, 0 to 85980
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   votes     85981 non-null  int64 
 1   headline  85981 non-null  object
 2   reviews   85981 non-null  object
 3   rating    85981 non-null  int64 
 4   target    85981 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 3.3+ MB


In [18]:
### import scikit learn libraries and split dataset
from sklearn.model_selection import train_test_split

In [19]:
train_ds, test_ds = train_test_split(reviews_df, test_size=0.25)

In [20]:
### import AutoNLP 
from autoviml.Auto_NLP import Auto_NLP

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt


Imported Auto_NLP version: 0.0.45.. Call using:
     train_nlp, test_nlp, nlp_pipeline, predictions = Auto_NLP(
                nlp_column, train, test, target, score_type='balanced_accuracy',
                modeltype='Classification',top_num_features=200, verbose=0,
                build_model=True)


In [21]:
nlp_column = 'reviews'
target = "target"

In [22]:
# print(nlp_column_name)
# print(target_column_name)

In [ ]:
train_nlp, test_nlp, nlp_transformer, preds = Auto_NLP(nlp_column,
                                                       train_ds,
                                                       target, 
                                                       test_ds, 
                                                       score_type='balanced_accuracy',
                                                       modeltype='Classification', 
                                                       top_num_features=100, 
                                                       verbose=1,
                                                       build_model=True)

Auto NLP processing on NLP Column: reviews
Shape of Train Data: 64485 rows
    Added 9 summary columns for counts of words and characters in each row
Cleaning text in Train data for reviews column
    Faster text processing using clean_text function, since top_num_features is below 300
    Time Taken for Expanding emojis and abbreviations in data = 127 seconds
        Time Taken for Processing text in data = 334 seconds
Train data Text cleaning completed. Time taken = 472 seconds

For target =        votes  \
16412      5   
12986      5   
12642      0   
6975       0   
76389      0   
...      ...   
27068      1   
7003       6   
22697      1   
64065      0   
12282      0   

                                                                    headline  \
16412                                                    b'Awesome Product!'   
12986                                                        b'Love Whisper'   
12642                               b'One Fantastic Item, Three Thro

In [1]:
nlp_transformer

NameError: ignored

In [ ]:
nlp_transformer.predict(test['nlp_column'])